<a href="https://colab.research.google.com/github/tvelichkovt/TensorFlow/blob/master/1_TensorFlow_%3E_math_expression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
    %tensorflow_version 2.x
except Exception:
  pass

In [31]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np

print(tf.__version__, keras.__version__)

2.1.0 2.2.4-tf


In [46]:
import tensorflow as tf
with tf.compat.v1.Session() as sess:
    hello = tf.constant('hello world')
    print(sess.run(hello))

b'hello world'


In [44]:
add1 = tf.add(3, 5)
sess = tf.Session()
print(sess.run(a))
sess.close()

AttributeError: ignored

In [27]:
#https://adventuresinmachinelearning.com/python-tensorflow-tutorial/
#https://adventuresinmachinelearning.com/tensorflow-eager-tutorial/

const = tf.constant(2.0, name="const")
b = tf.Variable(2.0, name='b')
c = tf.Variable(1.0, name='c')

b.read_value()
c.read_value()

<tf.Tensor: shape=(), dtype=float32, numpy=1.0>

In [0]:
# now create some operations
d = tf.add(b, c, name='d')
e = tf.add(c, const, name='e')
a = tf.multiply(d, e, name='a')

In [29]:
# setup the variable initialisation
init_op = tf.global_variables_initializer()

AttributeError: ignored

In [0]:
# start the session
with tf.Session() as sess:
    # initialise the variables
    sess.run(init_op)
    # compute the output of the graph
    a_out = sess.run(a)
    print("Variable a is {}".format(a_out))

Variable a is 9.0


In [0]:
# create TensorFlow variables
b = tf.placeholder(tf.float32, [None, 1], name='b')

In [0]:
a_out = sess.run(a, feed_dict={b: np.arange(0, 10)[:, np.newaxis]})
#https://stackoverflow.com/questions/44151611/how-can-i-solve-this-runtimeerror-attempted-to-use-a-closed-session
#error

RuntimeError: ignored